In [1]:
import pandas as pd
import numpy as np
import gzip
import math
import matplotlib.pyplot as plt
import seaborn as sns
import string
from collections import defaultdict
import regex as re
from dateutil.parser import parse

In [2]:
from fashionEDA import fashionCleaner

In [3]:
fc = fashionCleaner()

Scrapping Data, Null Entry: 1
Scrapping Data, Null Entry: 2
Scrapping Data, Null Entry: 3
Scrapping Data, Null Entry: 4
Scrapping Data, Null Entry: 5
Scrapping Data, Null Entry: 6
Scrapping Data, Null Entry: 7
Scrapping Data, Null Entry: 8
Scrapping Data, Null Entry: 9
Scrapping Data, Null Entry: 10
Scrapping Data, Null Entry: 11
Scrapping Data, Null Entry: 12
Scrapping Data, Null Entry: 13
Scrapping Data, Null Entry: 14
Scrapping Data, Null Entry: 15
Scrapping Data, Null Entry: 16
Scrapping Data, Null Entry: 17
Scrapping Data, Null Entry: 18
Scrapping Data, Null Entry: 19
Scrapping Data, Null Entry: 20
Scrapping Data, Null Entry: 21
Scrapping Data, Null Entry: 22
Scrapping Data, Null Entry: 23
Scrapping Data, Null Entry: 24
Scrapping Data, Null Entry: 25
Scrapping Data, Null Entry: 26
Scrapping Data, Null Entry: 27
Scrapping Data, Null Entry: 28
Scrapping Data, Null Entry: 29
Scrapping Data, Null Entry: 30
Scrapping Data, Null Entry: 31
Scrapping Data, Null Entry: 32
Scrapping Data, N

In [4]:
fashion_df = fc.getData()
fashion_df.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date,bust,cup
0,fit,420272,34d,2260466,137,10,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,68,14,28,2016-04-20,34,d
1,fit,273551,34b,153475,132,10,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,66,12,36,2013-06-18,34,b
2,fit,909926,34c,126335,135,8,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,65,8,34,2014-02-12,34,c
3,fit,151944,34b,616682,145,10,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,69,12,27,2016-09-26,34,b
4,fit,734848,32b,364092,138,8,date,Didn't actually wear it. It fit perfectly. The...,athletic,Traditional with a touch a sass,dress,68,8,45,2016-04-30,32,b


In [5]:
#tf-idf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [7]:
# Download the NLTK English tokenizer and the stopwords of all languages
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/brianhuang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brianhuang/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
words = " ".join(list(fashion_df['review_text'].values))
summary = " ".join(list(fashion_df['review_summary'].values))

In [11]:
words = words.split()
summary = summary.split()

In [12]:
punctuation = set(string.punctuation)

clean_words = [] 
for i in range(len(words)):
    clean_words.append(''.join([c for c in words[i].lower() if not c in punctuation]))

In [13]:
#added argument to this to include ngrams 
vectorizer = CountVectorizer(analyzer='word',ngram_range=(2, 2),
                        max_features = 10000,
                        tokenizer=word_tokenize,
                        stop_words=stopwords.words('english'))

In [14]:
#tfidf
tfidf = TfidfVectorizer(sublinear_tf=True,
                        analyzer='word',
                        max_features=1500,
                        tokenizer=word_tokenize)

In [15]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [16]:
# logisitic and linear models for prediction 
mod = LinearRegression()
mod2 = LogisticRegression()
mod_cup = LinearRegression()
mod2_cup = LogisticRegression()

In [17]:
as_is = ['weight', 'size', 'height', 'age', 'rating']
ohe = ['rented for', 'body type', 'category']
text_vec = ['review_summary']

In [18]:
#cahnged this to vectorizer instead of tfidf
text_transformer = make_column_transformer((vectorizer, 'review_summary'), remainder = 'passthrough')
#so we can use vectorizer on the review_summary

In [19]:

ct = ColumnTransformer(
    transformers = [("as_is", FunctionTransformer(lambda x: x), as_is),
    ("ohe", OneHotEncoder(handle_unknown = 'ignore'), ohe),
    ("vec", text_transformer, text_vec)]
)

In [20]:
#we use two models, linear and logisitic regression, both work alot better when we make the bust size ordinal rather than numerical
# i also did the same for cup prediction as well
pl = Pipeline(steps = [('transform', ct), ('Linregression', mod)])
pl2 = Pipeline(steps = [('transform', ct), ('Logregression', mod2)])
pl_cup = Pipeline(steps = [('transform', ct), ('Linregression', mod_cup)])
pl2_cup = Pipeline(steps = [('transform', ct), ('Logregression', mod2_cup)])

In [21]:
train, test = train_test_split(fashion_df, test_size = 0.2, random_state = 200)

In [22]:
def bust_category(bust):
    #encodes bust size into classes 1-10
    if bust == 28:
        return 0
    elif bust == 30:
        return 1
    elif bust == 32:
        return 2
    elif bust == 34:
        return 3
    elif bust == 36:
        return 4
    elif bust == 38:
        return 5
    elif bust == 40:
        return 6
    elif bust == 42:
        return 7
    elif bust == 44:
        return 8
    elif bust == 46:
        return 9
    else:
        return 10

In [23]:
def cup_category(cup):
     #encodes cup size into classes 1-12
    if cup == "aa":
        return 0
    elif cup == "a":
        return 1
    elif cup == "b":
        return 2
    elif cup == "c":
        return 3
    elif cup == "d":
        return 4
    elif cup == "d+":
        return 5
    elif cup == "dd":
        return 6
    elif cup == "ddd/e":
        return 7
    elif cup == "f":
        return 8
    elif cup == "g":
        return 9
    elif cup == "h":
        return 10
    elif cup == "i":
        return 11
    else:
        return 12

In [24]:
train["bust_cat"] = train["bust"].apply(bust_category)
test["bust_cat"] = test["bust"].apply(bust_category)
train["cup_cat"] = train["cup"].apply(cup_category)
test["cup_cat"] = test["cup"].apply(cup_category)

<ipython-input-24-37161a591f23>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["bust_cat"] = train["bust"].apply(bust_category)
<ipython-input-24-37161a591f23>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["bust_cat"] = test["bust"].apply(bust_category)
<ipython-input-24-37161a591f23>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [25]:
train_X, train_Y = train[['weight', 'size', 'height', 'age', 'rating', 'rented for', 'body type', 'category', 'review_summary']], train['bust_cat']
test_X, test_Y = test[['weight', 'size', 'height', 'age', 'rating', 'rented for', 'body type', 'category', 'review_summary']], test['bust_cat']

In [26]:
# training/test for X/Y but for cup categories
train_X, train_Y_cup = train[['weight', 'size', 'height', 'age', 'rating', 'rented for', 'body type', 'category', 'review_summary']], train['cup_cat']
test_X, test_Y_cup = test[['weight', 'size', 'height', 'age', 'rating', 'rented for', 'body type', 'category', 'review_summary']], test['cup_cat']

In [38]:
# fitting linear and logistic models for predicting cup size
# pl_cup.fit(train_X, train_Y_cup)
pl2_cup.fit(train_X, train_Y_cup)

/Users/brianhuang/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/brianhuang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('as_is',
                                                  FunctionTransformer(func=<function <lambda> at 0x7fa44d7301f0>),
                                                  ['weight', 'size', 'height',
                                                   'age', 'rating']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['rented for', 'body type',
                                                   'category']),
                                                 ('vec',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('countvectorizer',
                                                                                   CountVector...
       

In [39]:
# pl_cup.score(test_X,test_Y_cup) #linear model

In [40]:
pl2_cup.score(test_X,test_Y_cup)

0.35655975680568364

In [30]:
pl.fit(train_X, train_Y)

/Users/brianhuang/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('as_is',
                                                  FunctionTransformer(func=<function <lambda> at 0x7fa44d7301f0>),
                                                  ['weight', 'size', 'height',
                                                   'age', 'rating']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['rented for', 'body type',
                                                   'category']),
                                                 ('vec',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('countvectorizer',
                                                                                   CountVector...
       

In [31]:
preds = pl.predict(test_X)

In [32]:
preds = [round(pred) for pred in preds]

In [33]:
sum([1 if p == a else 0 for p, a in zip(preds, test_Y)])/len(preds)

0.571574956450456

In [34]:
mean_squared_error(preds, test_Y)

0.5006660518495748

In [35]:
pl2.fit(train_X, train_Y)

/Users/brianhuang/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/brianhuang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('as_is',
                                                  FunctionTransformer(func=<function <lambda> at 0x7fa44d7301f0>),
                                                  ['weight', 'size', 'height',
                                                   'age', 'rating']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['rented for', 'body type',
                                                   'category']),
                                                 ('vec',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('countvectorizer',
                                                                                   CountVector...
       

In [61]:
pl2.score(test_X, test_Y) #logistic regression on bust

0.5903610342589746

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [62]:
forest_cup = RandomForestClassifier(max_depth = 8)

In [63]:
pl_cup3 = Pipeline(steps = [('transform', ct), ('randforest', forest_cup)])

In [64]:
pl_cup3.fit(train_X, train_Y)

/Users/brianhuang/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('as_is',
                                                  FunctionTransformer(func=<function <lambda> at 0x7fa44d7301f0>),
                                                  ['weight', 'size', 'height',
                                                   'age', 'rating']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['rented for', 'body type',
                                                   'category']),
                                                 ('vec',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('countvectorizer',
                                                                                   CountVector...
       

In [65]:
pl_cup3.score(test_X, test_Y)

0.48246063462786487

In [72]:
forest_bust = RandomForestClassifier(max_depth = 8)

In [83]:
train_X, train_Y = train[['weight', 'size', 'height', 'age', 'rating', 'rented for', 'body type', 'category', 'review_summary']], train['bust_cat']
test_X, test_Y = test[['weight', 'size', 'height', 'age', 'rating', 'rented for', 'body type', 'category', 'review_summary']], test['bust_cat']

In [84]:
pl4 = Pipeline(steps = [('transform', ct), ('random_forest', forest_bust)])

In [88]:
pl4.fit(train_X, train_Y_cup)

/Users/brianhuang/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('as_is',
                                                  FunctionTransformer(func=<function <lambda> at 0x7fa44d7301f0>),
                                                  ['weight', 'size', 'height',
                                                   'age', 'rating']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['rented for', 'body type',
                                                   'category']),
                                                 ('vec',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('countvectorizer',
                                                                                   CountVector...
       

In [89]:
pl4.score(test_X, test_Y_cup)

0.3752775216039895

In [87]:
#logistic regression for bust size
#randomforestclassifier for cup size